# Handouts 3–4 demos (no FFT): integral intuition for the Fourier Transform

This notebook contains **live demos** designed to support Handout 3 (continuous spectrum, time–frequency trade-off) and Handout 4 (properties: shift, modulation, convolution, duality).

**Important:** we avoid FFTs for pedagogical reasons. All spectra are computed by **numerical quadrature** (Riemann / trapezoidal approximation of the Fourier integral).

---

## Conventions used here
We match your course convention:
\[
F(\omega)=\int_{-\infty}^{\infty} f(t)\,e^{-j\omega t}\,dt,\qquad
f(t)=\frac{1}{2\pi}\int_{-\infty}^{\infty} F(\omega)\,e^{j\omega t}\,d\omega.
\]

Since we cannot integrate to \(\pm\infty\) on a computer, we:
- choose a **time window** \([-T_\text{win}/2,\,T_\text{win}/2]\),
- sample time densely,
- approximate the integral with `np.trapz`.

This matches the “integral as a weighted sum” interpretation.

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---------- Utilities ----------

def ft_integral(f_t: np.ndarray, t: np.ndarray, w: np.ndarray) -> np.ndarray:
    # Numerical approximation of F(w) = ∫ f(t) e^{-j w t} dt
    expo = np.exp(-1j * w[:, None] * t[None, :])
    integrand = f_t[None, :] * expo
    return np.trapz(integrand, t, axis=1)

def ift_integral(F_w: np.ndarray, w: np.ndarray, t: np.ndarray) -> np.ndarray:
    # Numerical approximation of f(t) = (1/2π) ∫ F(w) e^{j w t} dw
    expo = np.exp(1j * w[None, :] * t[:, None])
    integrand = F_w[None, :] * expo
    return (1/(2*np.pi)) * np.trapz(integrand, w, axis=1)

def sinc(x):
    # numpy's sinc is sin(pi x)/(pi x); we want sin(x)/x
    out = np.ones_like(x, dtype=float)
    m = np.abs(x) > 1e-12
    out[m] = np.sin(x[m]) / x[m]
    return out

def setup_time(Twin=10.0, N=4001):
    t = np.linspace(-Twin/2, Twin/2, N)
    dt = t[1] - t[0]
    return t, dt

def setup_omega(Wmax=60.0, M=2001):
    w = np.linspace(-Wmax, Wmax, M)
    dw = w[1] - w[0]
    return w, dw

def mag_phase(F):
    return np.abs(F), np.unwrap(np.angle(F))

print("Ready.")


## Demo 1 — The FT as a rotating vector sum (phasor walk)

**Goal:** the integral is a sum of tiny complex vectors.

We compute partial integrals
\[
F(\omega; t_\text{end}) = \int_{-T_\text{win}/2}^{t_\text{end}} f(t)e^{-j\omega t}\,dt
\]
and plot the trajectory in the complex plane.

### Live questions
1. What does the path look like when \(\omega\) matches a frequency in the signal?
2. What changes if we double \(\omega\)?
3. Why does “wrong \(\omega\)” cause cancellation?


In [ ]:
t, dt = setup_time(Twin=12.0, N=6001)
w0 = 12.0
sigma = 1.2
f = np.cos(w0*t) * np.exp(-(t/sigma)**2)

def partial_F(f, t, w):
    expo = np.exp(-1j*w*t)
    integrand = f * expo
    Fp = np.cumsum((integrand[:-1] + integrand[1:]) * 0.5) * (t[1]-t[0])
    return np.concatenate([[0+0j], Fp])

w_match = w0
w_wrong = 2.4*w0

Fp_match = partial_F(f, t, w_match)
Fp_wrong = partial_F(f, t, w_wrong)

plt.figure()
plt.plot(Fp_match.real, Fp_match.imag, label=rf"$\omega={w_match:.1f}$ (match)")
plt.plot(Fp_wrong.real, Fp_wrong.imag, label=rf"$\omega={w_wrong:.1f}$ (mismatch)")
plt.axhline(0); plt.axvline(0)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("Re"); plt.ylabel("Im")
plt.title("Phasor walk: partial Fourier integral")
plt.legend()
plt.show()


## Demo 2 — Continuous spectrum from a finite-duration signal (rectangle → sinc)

**Goal:** finite-duration, non-periodic signals produce **continuous** spectra.

Signal:
\[
f(t)=\begin{cases}
1, & |t|<T/2\\
0, & \text{otherwise}
\end{cases}
\]
Theory (shape):
\[
F(\omega)=T\,\mathrm{sinc}\!\left(\frac{\omega T}{2}\right)
\quad\text{with }\mathrm{sinc}(x)=\sin x/x.
\]

### Live questions
1. Why isn’t the spectrum discrete?
2. What time-domain feature causes the sinc side-lobes?
3. What happens to the spectrum if we halve the pulse width?


In [ ]:
t, dt = setup_time(Twin=12.0, N=6001)
w, dw = setup_omega(Wmax=80.0, M=3001)

T_pulse = 2.0
f_rect = (np.abs(t) < T_pulse/2).astype(float)

F_num = ft_integral(f_rect, t, w)
F_theory = T_pulse * sinc(w*T_pulse/2)

plt.figure()
plt.plot(w, np.abs(F_num), label="|F(ω)| (numerical integral)")
plt.plot(w, np.abs(F_theory), "--", label="|F(ω)| (theory shape)")
plt.xlim(-60, 60)
plt.xlabel("ω"); plt.ylabel("|F(ω)|")
plt.title("Rectangle in time → sinc-like continuous spectrum")
plt.legend()
plt.show()


## Demo 3 — Time shift changes phase, not magnitude

Property:
\[
f(t-t_0)\ \longleftrightarrow\ F(\omega)e^{-j\omega t_0}.
\]

### Live questions
1. If you delay a signal in time, why shouldn’t its frequency content change?
2. Why are higher frequencies more sensitive to delay?
3. Can you estimate \(t_0\) from the phase slope?


In [ ]:
t, dt = setup_time(Twin=12.0, N=6001)
w, dw = setup_omega(Wmax=60.0, M=2001)

sigma = 1.0
f0 = np.exp(-(t/sigma)**2)

t0 = 0.9
f_shift = np.interp(t, t - t0, f0, left=0.0, right=0.0)

F0 = ft_integral(f0, t, w)
F1 = ft_integral(f_shift, t, w)

mag0, ph0 = mag_phase(F0)
mag1, ph1 = mag_phase(F1)

plt.figure()
plt.plot(w, mag0, label="|F(ω)| original")
plt.plot(w, mag1, "--", label="|F(ω)| shifted")
plt.xlabel("ω"); plt.ylabel("Magnitude")
plt.title("Magnitude unchanged by time shift")
plt.legend()
plt.show()

plt.figure()
plt.plot(w, ph1 - ph0, label="Δ phase")
plt.plot(w, -w*t0, "--", label="theory: -ω t0")
plt.xlabel("ω"); plt.ylabel("Phase (rad)")
plt.title("Time shift → linear phase ramp")
plt.legend()
plt.show()


## Demo 4 — Modulation shifts the spectrum (no FFT)

For cosine modulation:
\[
f(t)\cos(\omega_0 t)\ \longleftrightarrow\ \tfrac12F(\omega-\omega_0)+\tfrac12F(\omega+\omega_0).
\]

### Live questions
1. Why do we get *two* shifted copies?
2. What sets the shape of each copy?
3. How would you choose \(\omega_0\) to avoid overlap?


In [ ]:
t, dt = setup_time(Twin=12.0, N=6001)
w, dw = setup_omega(Wmax=80.0, M=3001)

sigma = 0.9
f = np.exp(-(t/sigma)**2)

w_c = 25.0
f_mod = f * np.cos(w_c*t)

F = ft_integral(f, t, w)
Fmod = ft_integral(f_mod, t, w)

plt.figure()
plt.plot(w, np.abs(F), label="|F(ω)| baseband")
plt.plot(w, np.abs(Fmod), label="|F_mod(ω)| after cos modulation")
plt.xlim(-80, 80)
plt.xlabel("ω"); plt.ylabel("Magnitude")
plt.title("Cosine modulation copies the spectrum to ±ωc")
plt.legend()
plt.show()


## Demo 5 — Convolution theorem (integral version)

We check:
\[
\mathcal{F}\{f*g\}(\omega)\approx F(\omega)G(\omega)
\]
where the convolution \(f*g\) is computed as a Riemann-sum approximation of the convolution integral.

### Live questions
1. Which is easier: convolution in time or multiplication in frequency?
2. If \(g\) is a smoothing kernel, what happens to high frequencies?
3. How does the width of \(g\) control the cutoff?


In [ ]:
t, dt = setup_time(Twin=14.0, N=7001)
w, dw = setup_omega(Wmax=60.0, M=2001)

f = np.exp(-(t/0.6)**2) * np.cos(18*t)

sigma_g = 0.5
g = np.exp(-(t/sigma_g)**2)
g = g / np.trapz(g, t)  # area 1

# Convolution integral approximated by discrete convolution with dt scaling
h = np.convolve(f, g, mode="same") * dt

F = ft_integral(f, t, w)
G = ft_integral(g, t, w)
H_from_time = ft_integral(h, t, w)
H_from_freq = F * G

plt.figure()
plt.plot(w, np.abs(H_from_time), label="|FT{f*g}|")
plt.plot(w, np.abs(H_from_freq), "--", label="|F·G|")
plt.xlabel("ω"); plt.ylabel("Magnitude")
plt.title("Convolution theorem check")
plt.legend()
plt.show()


## Optional — “Effect of convolution on sounds” (still no FFT)

We generate a short sound-like waveform (sum of tones), apply:
- an **echo** impulse response,
- a **smoothing** kernel (low-pass),
then estimate frequency content via a short-window Fourier **integral**.

### Live questions
1. What does echo do in time? What might it imply in frequency?
2. Why does smoothing remove “brightness” (high-frequency content)?
3. Where do you see the time–frequency trade-off?


In [ ]:
fs = 2000
dur = 0.8
t = np.linspace(0, dur, int(fs*dur), endpoint=False)
dt = t[1]-t[0]

x = (0.8*np.sin(2*np.pi*120*t) +
     0.5*np.sin(2*np.pi*260*t) +
     0.3*np.sin(2*np.pi*420*t))
x *= (1 - np.exp(-12*t))

delay = 0.08
h_echo = np.zeros_like(t)
h_echo[0] = 1.0
h_echo[int(delay*fs)] = 0.6

klen = int(0.05*fs)
n = np.arange(-klen, klen+1)
gk = np.exp(-(n/(0.012*fs))**2)
gk = gk / np.sum(gk)

y_echo = np.convolve(x, h_echo, mode="same")
y_smooth = np.convolve(x, gk, mode="same")

plt.figure()
plt.plot(t, x, label="original")
plt.plot(t, y_echo, label="with echo", alpha=0.9)
plt.xlim(0, 0.25)
plt.xlabel("t (s)"); plt.ylabel("amplitude")
plt.title("Time domain: echo")
plt.legend()
plt.show()

plt.figure()
plt.plot(t, x, label="original")
plt.plot(t, y_smooth, label="smoothed", alpha=0.9)
plt.xlim(0, 0.25)
plt.xlabel("t (s)"); plt.ylabel("amplitude")
plt.title("Time domain: smoothing")
plt.legend()
plt.show()

Twin = 0.25
mask = (t >= 0) & (t <= Twin)
tt = t[mask]

w = np.linspace(0, 2*np.pi*800, 1200)  # up to 800 Hz
X = ft_integral(x[mask], tt, w)
YE = ft_integral(y_echo[mask], tt, w)
YS = ft_integral(y_smooth[mask], tt, w)

plt.figure()
plt.plot(w/(2*np.pi), np.abs(X), label="original")
plt.plot(w/(2*np.pi), np.abs(YE), label="echo")
plt.plot(w/(2*np.pi), np.abs(YS), label="smoothed")
plt.xlim(0, 700)
plt.xlabel("frequency f (Hz)"); plt.ylabel("magnitude")
plt.title("Frequency content via integral (no FFT)")
plt.legend()
plt.show()


## Instructor deployment notes
- **Lecture:** Demo 1 → Demo 2 → Demo 4 → Demo 5 is a strong narrative.
- **Supervisions:** Demo 3 (phase ramp) is a nice property check.
- If asked “why not FFT?”: **FFT is implementation; today is meaning.**
